# TensorFlow Tutorial

In [1]:
## note we need to make sure these environment variables are set properly for tensorflow to properly make use of the GPU
# should be: /app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn
!echo $CUDNN_PATH
# should be: /usr/local/cuda/lib64:/usr/local/cuda/lib64:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn/lib:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/tensorrt_libs:/app/apps/rhel8/python-anaconda3/2023-04/lib::
!echo $LD_LIBRARY_PATH
# must include: /app/apps/rhel8/jupyter/2023-04/bin:/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/jupyterhub/bin:/common/bin:/usr/local/cuda/bin:/usr/localsys/bin:/usr/share/Modules/bin
!echo $PATH

/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn
/usr/local/cuda/lib64:/usr/local/cuda/lib64:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/nvidia/cudnn/lib:/app/apps/rhel8/jupyter/2023-04/lib/python3.10/site-packages/tensorrt_libs:/app/apps/rhel8/python-anaconda3/2023-04/lib::
/app/apps/rhel8/jupyter/2023-04/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/jupyterhub/bin:/common/bin:/usr/local/cuda/bin:/usr/localsys/bin:/usr/share/Modules/bin:/bin


In [2]:
import tensorflow as tf

2023-09-09 00:59:29.068762: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-09 00:59:29.124732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.13.0


In [4]:
from tensorflow import keras
from tensorflow.keras import layers, models, datasets
import time
from tensorflow.python.client import device_lib
from numba import cuda 

In [5]:
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU' or x.device_type == 'CPU']
get_available_devices()

2023-09-09 00:59:34.042961: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:0 with 14551 MB memory:  -> device: 0, name: Tesla P6, pci bus id: 0000:18:00.0, compute capability: 6.1


['/device:CPU:0', '/device:GPU:0']

In [6]:
# Download and prepare the CIFAR10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Create the convolutional base
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Add Dense layers on top
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

# Compile and train the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
# Fit the model on GPU
start_time = time.time()
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))
end_time = time.time()
print("Time to fit model on GPU: {:.2f} seconds".format(end_time - start_time))

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_images, test_labels)
print("test loss, test acc:", results)

2023-09-09 00:59:35.211901: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14551 MB memory:  -> device: 0, name: Tesla P6, pci bus id: 0000:18:00.0, compute capability: 6.1


Epoch 1/10


2023-09-09 00:59:37.661436: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-09 00:59:37.909555: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe54750d5b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-09 00:59:37.909594: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla P6, Compute Capability 6.1
2023-09-09 00:59:37.915060: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-09 00:59:38.067031: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1563/1563 [==============================] - 9s 4ms/step - loss: 1.5739 - accuracy: 0.4199 - val_loss: 1.2904 - val_accuracy: 0.5304
Epoch 2/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.2047 - accuracy: 0.5687 - val_loss: 1.1444 - val_accuracy: 0.5895
Epoch 3/10
1563/1563 [==============================] - 5s 3ms/step - loss: 1.0551 - accuracy: 0.6275 - val_loss: 1.0405 - val_accuracy: 0.6385
Epoch 4/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.9473 - accuracy: 0.6668 - val_loss: 1.0200 - val_accuracy: 0.6501
Epoch 5/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.8714 - accuracy: 0.6948 - val_loss: 1.0212 - val_accuracy: 0.6481
Epoch 6/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.8088 - accuracy: 0.7171 - val_loss: 0.9440 - val_accuracy: 0.6732
Epoch 7/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7597 - accuracy: 0.7352 - val_loss: 0.8552 - val_accuracy: 0.7039
Epo

In [7]:
# free up the GPU memory so other people/processes can use it
device = cuda.get_current_device()
device.reset()

In [8]:
# code that I wrote for the PDFF imputation project using a baseline_mlp model
!cd /home/craut/wkspce/craut_lfs/MRI_PDFF_imputation/fit_models/ && time python evaluate_model.py -dd ../cut_data_again/real_data/ -od output2 -m baseline_mlp -tr -te -i simple

Num GPUs Available:  1
### TRAINING ###
(34634, 248) (34634,)
Fitting Model on All Data
Epoch 1/100
217/217 [==============================] - 4s 5ms/step - loss: 26.0695 - root_mean_squared_error: 4.8467 - mean_absolute_error: 3.1503 - val_loss: 430.4765 - val_root_mean_squared_error: 20.6907 - val_mean_absolute_error: 20.3818
Epoch 2/100
217/217 [==============================] - 1s 4ms/step - loss: 18.5623 - root_mean_squared_error: 4.0388 - mean_absolute_error: 2.4722 - val_loss: 42.5595 - val_root_mean_squared_error: 6.3574 - val_mean_absolute_error: 4.9741
Epoch 3/100
217/217 [==============================] - 1s 4ms/step - loss: 16.9152 - root_mean_squared_error: 3.8560 - mean_absolute_error: 2.3968 - val_loss: 16.8469 - val_root_mean_squared_error: 3.8599 - val_mean_absolute_error: 2.4229
Epoch 4/100
217/217 [==============================] - 1s 4ms/step - loss: 16.2387 - root_mean_squared_error: 3.7925 - mean_absolute_error: 2.3214 - val_loss: 18.5317 - val_root_mean_squared_e